<a href="https://colab.research.google.com/github/puneat/Audio_Sentiment/blob/puneet/data_augmentation_1d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [ ]:
# Importing required libraries 
# Keras
import keras
from keras import regularizers
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model, model_from_json
from keras.layers import Dense, Embedding, LSTM
from keras.layers import Input, Flatten, Dropout, Activation, BatchNormalization
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.utils import np_utils, to_categorical
from keras.callbacks import ModelCheckpoint

# sklearn
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Other  
import librosa
import librosa.display
import json
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
import pandas as pd
import seaborn as sns
import glob 
import os
from tqdm import tqdm
import pickle
import IPython.display as ipd  # To play sound in the notebook

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
#########################
# Augmentation methods
#########################
def noise(data):
    """
    Adding White Noise.
    """
    # you can take any distribution from https://docs.scipy.org/doc/numpy-1.13.0/reference/routines.random.html
    noise_amp = 0.05*np.random.uniform()*np.amax(data)   # more noise reduce the value to 0.5
    data = data.astype('float64') + noise_amp * np.random.normal(size=data.shape[0])
    return data
    
def shift(data):
    """
    Random Shifting.
    """
    s_range = int(np.random.uniform(low=-5, high = 5)*1000)  #default at 500
    return np.roll(data, s_range)
    
def stretch(data, rate=0.8):
    """
    Streching the Sound. Note that this expands the dataset slightly
    """
    data = librosa.effects.time_stretch(data, rate)
    return data
    
def pitch(data, sample_rate):
    """
    Pitch Tuning.
    """
    bins_per_octave = 12
    pitch_pm = 2
    pitch_change =  pitch_pm * 2*(np.random.uniform())   
    data = librosa.effects.pitch_shift(data.astype('float64'), 
                                      sample_rate, n_steps=pitch_change, 
                                      bins_per_octave=bins_per_octave)
    return data
    
def dyn_change(data):
    """
    Random Value Change.
    """
    dyn_change = np.random.uniform(low=-0.5 ,high=7)  # default low = 1.5, high = 3
    return (data * dyn_change)
    
def speedNpitch(data):
    """
    Speed and Pitch Tuning.
    """
    # you can change low and high here
    length_change = np.random.uniform(low=0.8, high = 1)
    speed_fac = 1.2  / length_change # try changing 1.0 to 2.0 ... =D
    tmp = np.interp(np.arange(0,len(data),speed_fac),np.arange(0,len(data)),data)
    minlen = min(data.shape[0], tmp.shape[0])
    data *= 0
    data[0:minlen] = tmp[0:minlen]
    return data

####################################
# the confusion matrix heat map plot
####################################
def print_confusion_matrix(confusion_matrix, class_names, figsize = (10,7), fontsize=14):
    """Prints a confusion matrix, as returned by sklearn.metrics.confusion_matrix, as a heatmap.
    
    Arguments
    ---------
    confusion_matrix: numpy.ndarray
        The numpy.ndarray object returned from a call to sklearn.metrics.confusion_matrix. 
        Similarly constructed ndarrays can also be used.
    class_names: list
        An ordered list of class names, in the order they index the given confusion matrix.
    figsize: tuple
        A 2-long tuple, the first value determining the horizontal size of the ouputted figure,
        the second determining the vertical size. Defaults to (10,7).
    fontsize: int
        Font size for axes labels. Defaults to 14.
        
    Returns
    -------
    matplotlib.figure.Figure
        The resulting confusion matrix figure
    """
    df_cm = pd.DataFrame(
        confusion_matrix, index=class_names, columns=class_names, 
    )
    fig = plt.figure(figsize=figsize)
    try:
        heatmap = sns.heatmap(df_cm, annot=True, fmt="d")
    except ValueError:
        raise ValueError("Confusion matrix values must be integers.")
        
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=fontsize)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=fontsize)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

### **MALE**

In [ ]:
# lets pick up the meta-data that we got from our first part of the Kernel
ref = pd.read_csv("/gdrive/My Drive/Audio_files/male_df.csv")
ref.head()

,labels,source,path
0,male_disgust,SAVEE,/gdrive/My Drive/Audio_files/male_final/SAVEE/...
1,male_disgust,SAVEE,/gdrive/My Drive/Audio_files/male_final/SAVEE/...
2,male_disgust,SAVEE,/gdrive/My Drive/Audio_files/male_final/SAVEE/...
3,male_disgust,SAVEE,/gdrive/My Drive/Audio_files/male_final/SAVEE/...
4,male_disgust,SAVEE,/gdrive/My Drive/Audio_files/male_final/SAVEE/...


In [ ]:
# Note this takes a couple of minutes (~16 mins) as we're iterating over 4 datasets, and with augmentation  
df = pd.DataFrame(columns=['feature'])
df_noise = pd.DataFrame(columns=['feature'])
df_shift = pd.DataFrame(columns=['feature'])
#df_stretch = pd.DataFrame(columns=['feature'])
df_pitch = pd.DataFrame(columns=['feature'])
df_dyn_change = pd.DataFrame(columns=['feature'])
df_speedpitch = pd.DataFrame(columns=['feature'])
cnt = 0

# loop feature extraction over the entire dataset
for i in tqdm(ref.path):
    
    # first load the audio 
    X, sample_rate = librosa.load(i
                                  , res_type='kaiser_fast'
                                  ,duration=2.5
                                  ,sr=44100
                                  ,offset=0.5
                                 )

    # take mfcc and mean as the feature. Could do min and max etc as well. 
    mfccs = np.mean(librosa.feature.mfcc(y=X, 
                                        sr=np.array(sample_rate), 
                                        n_mfcc=13),
                    axis=0)
    
    df.loc[cnt] = [mfccs]   

    # random shifting (omit for now)
    # Stretch
    # pitch (omit for now)
    # dyn change
    
    # noise 
    aug = noise(X)
    aug = np.mean(librosa.feature.mfcc(y=aug, 
                                    sr=np.array(sample_rate), 
                                    n_mfcc=13),    
                  axis=0)
    df_noise.loc[cnt] = [aug]

    # speed pitch
    aug = speedNpitch(X)
    aug = np.mean(librosa.feature.mfcc(y=aug, 
                                    sr=np.array(sample_rate), 
                                    n_mfcc=13),    
                  axis=0)
    df_speedpitch.loc[cnt] = [aug]   

    # pitch
    aug = pitch(X, sample_rate)
    aug = np.mean(librosa.feature.mfcc(y=aug, 
                                    sr=np.array(sample_rate), 
                                    n_mfcc=13),    
                  axis=0)
    df_pitch.loc[cnt] = [aug] 

    # shift
    aug = shift(X)
    aug = np.mean(librosa.feature.mfcc(y=aug, 
                                    sr=np.array(sample_rate), 
                                    n_mfcc=13),    
                  axis=0)
    df_shift.loc[cnt] = [aug]  

    # dyn_change
    aug = dyn_change(X)
    aug = np.mean(librosa.feature.mfcc(y=aug, 
                                    sr=np.array(sample_rate), 
                                    n_mfcc=13),    
                  axis=0)
    df_dyn_change.loc[cnt] = [aug]  

    # stretch
    # aug = stretch(X)
    # aug = np.mean(librosa.feature.mfcc(y=aug, 
    #                                 sr=np.array(sample_rate), 
    #                                 n_mfcc=13),    
    #               axis=0)
    # df_stretch.loc[cnt] = [aug]    

    cnt += 1

df.head()

Streaming output truncated to the last 5000 lines.
100%|██████████| 5130/5130 [37:14<00:00,  2.30it/s]


,feature
0,"[-17.685801316025962, -10.543401050054786, -7...."
1,"[-25.302001090966527, -24.328178594639958, -24..."
2,"[-26.13415153519962, -23.928130688466254, -23...."
3,"[-24.981073994339873, -23.702245772802055, -24..."
4,"[-27.752264740004097, -25.923429036142586, -24..."


In [ ]:
# combine 
df = pd.concat([ref,pd.DataFrame(df['feature'].values.tolist())],axis=1)
df_noise = pd.concat([ref,pd.DataFrame(df_noise['feature'].values.tolist())],axis=1)
df_pitch = pd.concat([ref,pd.DataFrame(df_pitch['feature'].values.tolist())],axis=1)
df_dyn_change = pd.concat([ref,pd.DataFrame(df_dyn_change['feature'].values.tolist())],axis=1)
df_shift = pd.concat([ref,pd.DataFrame(df_shift['feature'].values.tolist())],axis=1)
#df_stretch = pd.concat([ref,pd.DataFrame(df_stretch['feature'].values.tolist())],axis=1)
df_speedpitch = pd.concat([ref,pd.DataFrame(df_speedpitch['feature'].values.tolist())],axis=1)
print(df.shape,df_noise.shape,df_speedpitch.shape, df_pitch.shape, df_dyn_change.shape, df_shift.shape)

(5130, 219) (5130, 219) (5130, 219) (5130, 219) (5130, 219) (5130, 219)


In [ ]:
df = pd.concat([df, df_noise, df_speedpitch, df_pitch, df_dyn_change, df_shift],axis=0,sort=False)
df=df.fillna(0)
del df_noise, df_speedpitch
df.head()

,labels,source,path,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,...,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215
0,male_disgust,SAVEE,/gdrive/My Drive/Audio_files/male_final/SAVEE/...,-17.685801,-10.543401,-7.518024,-7.506939,-7.403337,-7.362275,-8.112685,-6.900735,-7.539663,-7.677543,-8.266721,-9.299829,-10.524870,-15.049426,-19.683942,-19.050067,-18.667913,-20.895418,-23.388832,-23.887907,-25.020802,-26.581469,-20.918649,-14.350466,-11.566920,-9.683457,-9.573993,-10.763644,-11.851552,-13.374164,-16.865402,-20.377562,-23.882169,-25.583794,-24.624932,-25.183511,-24.761721,...,-9.427887,-8.050047,-6.875153,-6.847067,-7.501703,-11.273141,-15.233299,-17.738540,-21.600643,-23.349737,-22.254533,-23.589939,-25.336841,-25.831961,-27.366389,-26.924206,-26.831335,-27.003823,-24.906468,-24.816904,-25.608896,-25.390564,-27.034643,-26.252514,-26.358090,-27.220914,-28.279947,-26.211946,-25.694268,-25.985311,-25.768369,-24.080482,-23.160257,-23.539012,-23.789284,-25.096969,-24.103728,-22.289075,-23.330710,-23.340112
1,male_disgust,SAVEE,/gdrive/My Drive/Audio_files/male_final/SAVEE/...,-25.302001,-24.328179,-24.939580,-24.932921,-15.922182,-9.669080,-9.113287,-10.833799,-10.096997,-8.566569,-8.906033,-7.871860,-9.418822,-10.009119,-10.193020,-10.780454,-10.459806,-10.747662,-11.497913,-11.451850,-11.062868,-10.295214,-10.414422,-10.493700,-10.004488,-11.568353,-12.606366,-12.271651,-11.122537,-10.865031,-10.775224,-11.461333,-13.333211,-14.883215,-17.008620,-20.033215,-20.588205,...,-7.462429,-7.808964,-7.672180,-8.076372,-7.451515,-7.493429,-7.403017,-7.776670,-6.474336,-5.761596,-5.537273,-4.615541,-4.275374,-5.860197,-6.728284,-8.172176,-9.817752,-10.770730,-12.146788,-13.995520,-16.950673,-18.689850,-20.579320,-23.219597,-24.630404,-24.744938,-16.205772,-9.773642,-7.804234,-8.606250,-9.588616,-10.779072,-11.047021,-10.245645,-9.663476,-10.854288,-12.546134,-11.642432,-8.647266,-5.034869
2,male_disgust,SAVEE,/gdrive/My Drive/Audio_files/male_final/SAVEE/...,-26.134152,-23.928131,-23.992998,-23.687232,-26.934288,-24.934039,-23.083398,-22.089126,-21.178202,-22.035588,-22.058052,-19.109700,-10.477399,-7.028228,-7.245320,-8.266217,-9.508348,-11.043246,-14.196529,-14.606633,-14.755610,-14.234541,-13.836773,-13.709015,-11.730912,-9.817653,-9.181870,-9.086506,-9.177656,-7.936095,-9.395794,-14.276868,-18.127514,-19.903403,-20.081414,-20.061916,-20.509236,...,-4.992591,-5.144972,-5.176300,-5.508100,-6.291518,-6.860570,-7.429581,-7.177023,-6.804778,-5.958700,-5.029949,-4.818337,-5.642507,-6.503660,-6.522467,-5.039698,-3.914023,-3.551429,-5.201806,-9.454711,-13.561260,-15.706702,-18.843068,-22.448579,-24.795962,-26.733256,-25.171927,-25.426535,-25.577378,-22.829461,-12.367325,-10.398124,-9.959911,-11.010429,-10.773854,-11.549666,-11.601620,-11.788002,-8.063619,-4.384248
3,male_disgust,SAVEE,/gdrive/My Drive/Audio_files/male_final/SAVEE/...,-24.981074,-23.702246,-24.368539,-23.824286,-20.085265,-20.491522,-21.797198,-20.689136,-14.338272,-11.482922,-10.521439,-12.182463,-11.595602,-12.030412,-12.058803,-11.096145,-11.726163,-12.529537,-11.746328,-12.215391,-11.619400,-9.801078,-9.414315,-9.077748,-8.705984,-9.851240,-13.996290,-19.624007,-20.968213,-22.996743,-25.072632,-24.864131,-23.185258,-21.814228,-21.784198,-23.577666,-19.323375,...,-16.178072,-9.391094,-8.730066,-7.953388,-7.110306,-6.925129,-8.492684,-9.398223,-8.185889,-6.323172,-5.538038,-8.082038,-12.421711,-16.895500,-20.280909,-21.456681,-21.099407,-18.585644,-17.797974,-19.957557,-23.856652,-24.464547,-24.439857,-24.845373,-25.096150,-21.645310,-14.021650,-11.428354,-12.111615,-11.454638,-11.261946,-12.151451,-13.157601,-13.586169,-14.426998,-15.093403,-13.683900,-13.934830,-9.288823,-4.696516
4,male_disgust,SAVEE,/gdrive/My Drive/Audio_files/male_final/SAVEE/...,-27.752265,-25.923429,-24.471903,-25.061312,-25.652226

In [ ]:
df.to_csv("/gdrive/My Drive/Audio_files/augmented_male.csv")

### **FEMALE**

In [ ]:
# lets pick up the meta-data that we got from our first part of the Kernel
ref = pd.read_csv("/gdrive/My Drive/Audio_files/female_df.csv")
ref.head()

,labels,source,path
0,female_fear,TESS,/gdrive/My Drive/Audio_files/female_final/TESS...
1,female_fear,TESS,/gdrive/My Drive/Audio_files/female_final/TESS...
2,female_fear,TESS,/gdrive/My Drive/Audio_files/female_final/TESS...
3,female_fear,TESS,/gdrive/My Drive/Audio_files/female_final/TESS...
4,female_fear,TESS,/gdrive/My Drive/Audio_files/female_final/TESS...


In [ ]:
# Note this takes a couple of minutes (~16 mins) as we're iterating over 4 datasets, and with augmentation  
df = pd.DataFrame(columns=['feature'])
df_noise = pd.DataFrame(columns=['feature'])
df_shift = pd.DataFrame(columns=['feature'])
#df_stretch = pd.DataFrame(columns=['feature'])
df_pitch = pd.DataFrame(columns=['feature'])
df_dyn_change = pd.DataFrame(columns=['feature'])
df_speedpitch = pd.DataFrame(columns=['feature'])
cnt = 0

# loop feature extraction over the entire dataset
for i in tqdm(ref.path):
    
    # first load the audio 
    X, sample_rate = librosa.load(i
                                  , res_type='kaiser_fast'
                                  ,duration=2.5
                                  ,sr=44100
                                  ,offset=0.5
                                 )

    # take mfcc and mean as the feature. Could do min and max etc as well. 
    mfccs = np.mean(librosa.feature.mfcc(y=X, 
                                        sr=np.array(sample_rate), 
                                        n_mfcc=13),
                    axis=0)
    
    df.loc[cnt] = [mfccs]   

    # random shifting (omit for now)
    # Stretch
    # pitch (omit for now)
    # dyn change
    
    # noise 
    aug = noise(X)
    aug = np.mean(librosa.feature.mfcc(y=aug, 
                                    sr=np.array(sample_rate), 
                                    n_mfcc=13),    
                  axis=0)
    df_noise.loc[cnt] = [aug]

    # speed pitch
    aug = speedNpitch(X)
    aug = np.mean(librosa.feature.mfcc(y=aug, 
                                    sr=np.array(sample_rate), 
                                    n_mfcc=13),    
                  axis=0)
    df_speedpitch.loc[cnt] = [aug]   

    # pitch
    aug = pitch(X, sample_rate)
    aug = np.mean(librosa.feature.mfcc(y=aug, 
                                    sr=np.array(sample_rate), 
                                    n_mfcc=13),    
                  axis=0)
    df_pitch.loc[cnt] = [aug] 

    # shift
    aug = shift(X)
    aug = np.mean(librosa.feature.mfcc(y=aug, 
                                    sr=np.array(sample_rate), 
                                    n_mfcc=13),    
                  axis=0)
    df_shift.loc[cnt] = [aug]  

    # dyn_change
    aug = dyn_change(X)
    aug = np.mean(librosa.feature.mfcc(y=aug, 
                                    sr=np.array(sample_rate), 
                                    n_mfcc=13),    
                  axis=0)
    df_dyn_change.loc[cnt] = [aug]  

    # stretch
    # aug = stretch(X)
    # aug = np.mean(librosa.feature.mfcc(y=aug, 
    #                                 sr=np.array(sample_rate), 
    #                                 n_mfcc=13),    
    #               axis=0)
    # df_stretch.loc[cnt] = [aug]    

    cnt += 1

df.head()

Streaming output truncated to the last 5000 lines.
 99%|█████████▊| 6932/7029 [1:17:32<01:15,  1.29it/s]

In [ ]:
# combine 
df = pd.concat([ref,pd.DataFrame(df['feature'].values.tolist())],axis=1)
df_noise = pd.concat([ref,pd.DataFrame(df_noise['feature'].values.tolist())],axis=1)
df_pitch = pd.concat([ref,pd.DataFrame(df_pitch['feature'].values.tolist())],axis=1)
df_dyn_change = pd.concat([ref,pd.DataFrame(df_dyn_change['feature'].values.tolist())],axis=1)
df_shift = pd.concat([ref,pd.DataFrame(df_shift['feature'].values.tolist())],axis=1)
#df_stretch = pd.concat([ref,pd.DataFrame(df_stretch['feature'].values.tolist())],axis=1)
df_speedpitch = pd.concat([ref,pd.DataFrame(df_speedpitch['feature'].values.tolist())],axis=1)
print(df.shape,df_noise.shape,df_speedpitch.shape, df_pitch.shape, df_dyn_change.shape, df_shift.shape,df_stretch.shape)


(7029, 219) (7029, 219) (7029, 219) (7029, 219) (7029, 219) (7029, 219) (7029, 273)


In [ ]:
df = pd.concat([df, df_noise, df_speedpitch,df_pitch, df_dyn_change, df_shift],axis=0,sort=False)
df=df.fillna(0)
del df_noise, df_speedpitch
df.head()

,labels,source,path,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,...,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215
0,female_fear,TESS,/gdrive/My Drive/Audio_files/female_final/TESS...,-18.165691,-22.280521,-32.869496,-33.348384,-31.442652,-31.226946,-32.970442,-33.405853,-35.146622,-35.080820,-32.162143,-32.314888,-33.737759,-33.069504,-31.279293,-31.751031,-31.841785,-31.157676,-30.931237,-30.124957,-29.782879,-32.158049,-32.337431,-30.675274,-31.730434,-34.623561,-35.976177,-37.867385,-38.295264,-38.250943,-39.129616,-38.524891,-39.097919,-39.864353,-38.091470,-37.163623,-35.982447,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,female_fear,TESS,/gdrive/My Drive/Audio_files/female_final/TESS...,-15.906002,-20.206035,-36.018307,-35.941433,-33.626322,-31.863684,-34.244930,-32.479571,-32.478172,-33.285993,-34.209077,-36.464251,-36.647254,-35.330317,-34.510329,-33.729459,-32.513479,-33.482710,-35.434165,-35.072211,-33.458010,-33.214199,-34.357680,-33.464561,-32.515809,-31.309234,-33.015302,-35.088208,-33.007208,-31.246055,-29.818128,-28.506141,-28.294383,-31.134243,-33.071401,-34.568117,-35.536651,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,female_fear,TESS,/gdrive/My Drive/Audio_files/female_final/TESS...,-35.817310,-33.259054,-34.746449,-37.156902,-35.278470,-35.749107,-34.313131,-32.307477,-31.753337,-31.757874,-32.536862,-33.188495,-33.653722,-34.297734,-34.124224,-32.213580,-33.118894,-34.356938,-34.071390,-34.237020,-33.910608,-33.379212,-33.951564,-32.991469,-31.926951,-31.510142,-31.347825,-33.509127,-36.204372,-36.973021,-35.409051,-30.808756,-30.061974,-31.426117,-32.489891,-33.225080,-34.988262,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,female_fear,TESS,/gdrive/My Drive/Audio_files/female_final/TESS...,-16.569900,-20.540696,-31.054675,-30.003052,-29.991817,-31.160669,-33.608597,-35.381098,-37.277302,-35.547838,-32.533438,-32.138186,-31.699817,-32.305732,-32.739847,-31.672572,-30.828298,-32.681893,-31.391616,-28.571363,-29.069535,-33.257574,-32.603988,-31.257130,-32.009569,-36.388628,-40.161109,-40.646639,-40.015500,-40.219691,-37.696075,-34.306881,-32.456394,-30.557606,-27.925781,-22.087268,-20.069869,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,female_fear,TESS,/gdrive/My Drive/Audio_files/female_final/TESS...,-14.927656,-19.177805,-32.440822,-32.021896,-32.209691,-34.273317,-35.611009,-35.120896,-32.919323,-32.953254,-33.469782,-34.013599,-35.458182,-36.381816,-36.383833,-36.746485,-35.774530,-35.460933,-34.599711,-33.887741,-34.201649,-35.844957,-34.925647,-33.970445,-34.496503,-34.793064,-34.780491,-36.446122,-35.125235,-33.393734,-33.592417,-32.474319,-32.628319,-34.141552,-36.960046,-40.569463,-43.847191,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df.to_csv("/gdrive/My Drive/Audio_files/augmented_female.csv",index=False)